In [2]:
import time
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

batch_size=1000
test_batch_size=1000
epochs=1000
device=torch.device("cpu")
device=torch.device("cuda:0")
data_path='../../../data'
kwargs = {'num_workers': 1, 'pin_memory': True} if device != torch.device('cpu') else {}
train_loader = torch.utils.data.DataLoader( 
    datasets.MNIST(data_path, train=True,download=True, 
    transform=transforms.Compose([ transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) ])), batch_size=batch_size,
    shuffle=False, **kwargs)
test_loader = torch.utils.data.DataLoader(datasets.MNIST(data_path, train=False,
    transform=transforms.Compose([ transforms.ToTensor(), transforms.Normalize((0.1307,),
    (0.3081,)) ])), batch_size=test_batch_size, shuffle=False, **kwargs)
train_data=[]
test_data=[]
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    train_data.append((data,target))
for batch_idx, (data, target) in enumerate(test_loader):
    data, target = data.to(device), target.to(device)
    test_data.append((data,target))

In [18]:
n_in=784
n_out=10
n_hidden=500
model = "lr" # logistic regression
model = "fc2" # 2-layer MLP
model = "conv" # convolution
if (model == "lr"):
    net = nn.Sequential(nn.Linear(n_in,n_out))
elif(model == "fc2"):
    net = nn.Sequential(nn.Linear(n_in,n_hidden),nn.Sigmoid(),nn.Linear(n_hidden,n_out))
elif(model == "conv"):
    net = nn.Sequential(nn.Conv2d(1,20,5),nn.ReLU(),nn.MaxPool2d(2),nn.Conv2d(20,50,5),nn.ReLU(),nn.MaxPool2d(2))
    net2 = nn.Sequential(nn.Linear(4*4*50,n_hidden),nn.ReLU(),nn.Linear(n_hidden,n_out))
else:
    print("Wrong model")
net = net.to(device)
net2 = net2.to(device)
optimizer = torch.optim.Adam(list(net.parameters()), lr=0.001, betas=(0.9, 0.999))
print_bin = epochs/100
for epoch in range(1, epochs + 1):
    optimizer.zero_grad()
    t1=time.time()
    for batch_idx, (data, target) in enumerate(train_data):
        if(model == 'conv'):
            output=net(data).view(output.shape[0],-1)
            output = net2(output)
        else:
            output=net(data.view(data.shape[0],-1))

        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
    test_loss = 0
    correct = 0
    for data, target in test_data:
        if(model == 'conv'):
            output=net(data).view(output.shape[0],-1)
            output = net2(output)
        else:
            output=net(data.view(data.shape[0],-1))
        output=F.log_softmax(output, dim=1)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= 10000
    correct /= 10000
    if(epoch % print_bin==0):
        print("#%d loss=%.3f test_loss=%.3f accuracy=%.3f time=%.3f"%(epoch,loss,test_loss,correct,time.time()-t1))
        

#10 loss=0.533 test_loss=0.594 accuracy=0.851 time=0.419
#20 loss=0.466 test_loss=0.483 accuracy=0.837 time=0.421
#30 loss=0.213 test_loss=0.250 accuracy=0.933 time=0.419
#40 loss=0.213 test_loss=0.244 accuracy=0.928 time=0.422
#50 loss=0.179 test_loss=0.200 accuracy=0.943 time=0.419
#60 loss=0.141 test_loss=0.154 accuracy=0.956 time=0.419
#70 loss=0.186 test_loss=0.195 accuracy=0.942 time=0.420
#80 loss=0.123 test_loss=0.125 accuracy=0.965 time=0.420
#90 loss=0.139 test_loss=0.136 accuracy=0.960 time=0.422
#100 loss=0.109 test_loss=0.108 accuracy=0.970 time=0.420
#110 loss=0.105 test_loss=0.098 accuracy=0.972 time=0.419
#120 loss=0.228 test_loss=0.269 accuracy=0.907 time=0.419
#130 loss=0.102 test_loss=0.088 accuracy=0.975 time=0.420
#140 loss=0.098 test_loss=0.083 accuracy=0.976 time=0.419
#150 loss=0.246 test_loss=0.209 accuracy=0.936 time=0.420
#160 loss=0.095 test_loss=0.077 accuracy=0.978 time=0.421
#170 loss=0.092 test_loss=0.073 accuracy=0.978 time=0.419
#180 loss=0.090 test_lo

KeyboardInterrupt: 